# Tutorial 1 - Photon redirection

This tutorial demonstrates the very basic working principles of PennyLane for continuous-variable backends. ...




## Imports

First we need to import openqml, as well as openqml's version of numpy. This allows us to automatically compute gradients for functions that manipulate numpy arrays, including quantum functions.

In [18]:
import openqml as qm
from openqml import numpy as np
from openqml._optimize import GradientDescentOptimizer, AdagradOptimizer

Next, create a "device" to run the quantum node. 

In [2]:
dev1 = qm.device('default.qubit', wires=1)

## Quantum function

We define a quantum function called "circuit". 

In [3]:
@qm.qfunc(dev1)
def circuit(weights):
    
    qm.RX(weights[0], [0])
    qm.RY(weights[1], [0])
    
    return qm.expectation.PauliZ(0)

This function uses openqml to run the following quantum circuit:

![rotation circuit](figures/rotation_circuit.png "")

Starting with a qubit in the ground state, 

$$ |0\rangle = \begin{pmatrix}1 \\ 0 \end{pmatrix}, $$

we first rotate the qubit around the x-axis by 
$$R_x(w_0) = e^{-iw_0 X /2} = 
\begin{pmatrix} \cos \frac{w_0}{2} &  -i \sin \frac{w_0}{2} \\  
                -i \sin \frac{w_0}{2} &  \cos \frac{w_0}{2} 
\end{pmatrix}, $$ 
               
and then around the y-axis by 
$$ R_y(w_1) = e^{-i w_1 Y/2} = 
\begin{pmatrix} \cos \frac{w_1}{2} &  - \sin \frac{w_1}{2} \\  
                \sin \frac{w_1}{2} &  \cos \frac{w_1}{2} 
\end{pmatrix}. $$ 

After these operations the qubit is in the state

$$ | \psi \rangle = R_y(w_0) R_x(w_1) | 0 \rangle $$

Finally, we measure the expectation $ \langle \psi | Z | \psi \rangle $ of the Pauli-Z operator 
$$Z = 
\begin{pmatrix} 1 &  0 \\  
                0 & -1 
\end{pmatrix}. $$ 


Depending on the circuit parameters $w_0$ and $w_1$, the output expectation lies between $1$ (if $| \psi \rangle = | 0  \rangle $) and $-1$ (if $| \psi \rangle = | 1  \rangle $).

## Objective

Next, we define a cost. Here, the cost is directly the expectation of the PauliZ measurement, so that the cost is trivially the output of the circuit.

In [20]:
def objective(weights):
    return circuit(weights)

With this objective, the optimization procedure is supposed to find the weights that rotate the qubit from the ground state 

 <img src="figures/bloch_before.png" width="250"> 
 
 to the excited state
 
 <img src="figures/bloch_after.png" width="250">
 
 The rotation gates give the optimization landscape a trigonometric shape with four global minima and five global maxima.
 
 <img src="figures/optlandscape.png" width="450">

 
 

## Optimization

The initial values of the x- and y-rotation parameters are set to near-zero. This corresponds to identity gates, in other words, the circuit leaves the qubit in the ground state. *Note that at zero exactly the gradient is zero and the optimization algorithm will not descent from the maximum.*

In [6]:
weights0 = np.array([0.01, 0.01])
print('Initial rotation angles:', weights0)

Initial rotation angles: [0.01 0.01]


We choose a simple Gradient Descent Optimizer and update the weights for 10 steps.

In [21]:
o = GradientDescentOptimizer(0.5)

weights = weights0
for step in np.arange(1, 101):
    weights = o.step(objective, weights)
    if step%5==0:
        print('Cost after step {}: {}'.format(step, cost(weights)) )

print()
print('Optimized rotation angles:', weights)

Cost after step 5: 0.9942561248007442
Cost after step 10: 0.7312843497373493
Cost after step 15: 0.008145934071862121
Cost after step 20: 8.09318002101267e-06
Cost after step 25: 7.903630999672728e-09
Cost after step 30: 7.718409245072166e-12
Cost after step 35: 7.66053886991358e-15
Cost after step 40: 3.3306690738754696e-16
Cost after step 45: 1.6653345369377348e-16
Cost after step 50: -1.3877787807814457e-16
Cost after step 55: -8.798517470154366e-15
Cost after step 60: -5.035138972431241e-13
Cost after step 65: -2.903546847399241e-11
Cost after step 70: -1.6743259834139934e-09
Cost after step 75: -9.655006341646732e-08
Cost after step 80: -5.567542194023778e-06
Cost after step 85: -0.0003209827748970606
Cost after step 90: -0.01827993232467756
Cost after step 95: -0.5894463605133992
Cost after step 100: -0.9988743067653387

Optimized rotation angles: [-0.03355765  3.108035  ]


Starting at a different offset, we train another optimizer called Adagrad, which improves on gradient descent.

In [19]:
weights0 = np.array([-0.01, 0.01])
print('Initial rotation angles:', weights0)

o = AdagradOptimizer(0.5)

weights = weights0
for step in np.arange(1, 101):
    weights = o.step(cost, weights)
    if step%5==0:
        print('Cost after step {}: {}'.format(step, cost(weights)) )

print()
print('Optimized rotation angles:', weights)

Initial rotation angles: [-0.01  0.01]
Cost after step 5: 0.00013306617779057506
Cost after step 10: 6.098715976676772e-11
Cost after step 15: 5.551115123125783e-17
Cost after step 20: 1.1102230246251565e-16
Cost after step 25: 1.6653345369377348e-16
Cost after step 30: -1.6653345369377348e-16
Cost after step 35: -2.7755575615628914e-16
Cost after step 40: -7.119305145408816e-14
Cost after step 45: -2.120639774894073e-11
Cost after step 50: -6.3168203168206816e-09
Cost after step 55: -1.881604829112593e-06
Cost after step 60: -0.000560067861131136
Cost after step 65: -0.13763076512457606
Cost after step 70: -0.9702268349223584
Cost after step 75: -0.9998837262003906
Cost after step 80: -0.9999995647190025
Cost after step 85: -0.9999999983707548
Cost after step 90: -0.9999999999939022
Cost after step 95: -0.9999999999999776
Cost after step 100: -1.0

Optimized rotation angles: [-9.24310272e-09  3.14159264e+00]


 Adagrad and gradient descent find the same minimum, and, since neither has information on second order derivatives, both take a detour through a saddle point. However, Adagrad takes considerably fewer steps.
 
 <img src="figures/gd_vs_adag.png" width="450">